In [1]:
from numpy import float32,float64,log2
import sys
from matplotlib import pyplot as plt
import math
import numpy as np
import random
import scipy
import mpmath as mp
from mpmath import mpf, nstr, nprint
%matplotlib notebook

print("Wersja Pythona:")
print (sys.version)
print(f"Wersja numpy: {np.__version__}")
print("Konfiguracja liczb zmiennoprzecinkowych")
print(sys.float_info)

f64 = np.float64
f32 = np.float32


Wersja Pythona:
3.9.4 (tags/v3.9.4:1f2e308, Apr  6 2021, 13:40:21) [MSC v.1928 64 bit (AMD64)]
Wersja numpy: 1.20.2
Konfiguracja liczb zmiennoprzecinkowych
sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


In [2]:
def f_1(x): return mp.cos(x) * mp.cosh(x) - mpf(1)
def f_2(x): return mpf(1) / mpf(x) - mp.tan(x)
def f_3(x): return mpf(2) ** mpf(-x) + mp.exp(x) + mpf(2) * mp.cos(x) - mpf(6)

In [3]:
def f1(): return f_1, mp.pi * mpf(3) / mpf(2), mpf(2) * mp.pi
def f2(): return f_2, mpf('1e-20'), mp.pi / mpf(2) - mpf('1e-20')
def f3(): return f_3, mpf(1), mpf(3)


In [4]:
def bisection(f, prec, l, r, epsilon):
    steps = 1
    if(f(l) * f(r) > 0):
        print('Bad interval')
        return None
    xs = []
    with mp.workdps(prec):  
        while (r - l) / mpf(2) > epsilon:
            steps += 1
            m = l + (r - l) / mpf(2)
            xs.append(m)
            f_m = f(m)

            # print(f'{nstr(f(l), 3)}, {nstr(f_m, 3)}, {nstr(f(r), 3)}')

            if f(l) * f_m < mpf(0):
                r = m
            elif f(r) * f_m:
                l = m
            else:
                return m, f(m), steps, xs
        m = l + (r - l) / mpf(2)
        xs.append(m)
        return m, f(m), steps, xs


In [5]:
from math import floor, log, ceil


In [6]:
import pandas as pd
from itertools import product

data = []
precision = 50
    
for fun, eps in product ([f1, f2, f3], [-7, -15, -33]):
    f, l, r = fun()
    epsilon = mpf(f'1e{eps}')
    x, f_x, steps, _ = bisection(f, precision, l, r, epsilon)
    prec = ceil(- log(epsilon) / log(10))
    data.append([str(fun.__name__), f'1e{eps}', nstr(x, prec), nstr(f_x, 3), str(steps)])

pd.DataFrame(data, columns=['Name', 'Epsilon', 'x', 'f(x)', 'Steps'])

,Name,Epsilon,x,f(x),Steps
0,f1,1e-7,4.730041,-1.79e-6,24
1,f1,1e-15,4.7300407448627,-8.44e-15,51
2,f1,1e-33,4.73004074486270402602404810083388,7.58e-33,111
3,f2,1e-7,0.8603336,1.24e-7,24
4,f2,1e-15,0.86033358901938,9.4e-16,51
5,f2,1e-33,0.860333589019379762483893424137662,1.3e-33,111
6,f3,1e-7,1.829384,-2.04e-7,25
7,f3,1e-15,1.82938360193385,2.44e-15,51
8,f3,1e-33,1.82938360193384881713621294681415,8.7e-34,111
